In [3]:
!pip install bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [bs4]


In [4]:
import requests
from bs4 import BeautifulSoup
import re

## Obtener los links a scrapear

En esta parte del codigo se obtienen solamente los links que en la siguiente celda vamos a scrapear. 

In [ ]:
base_url = "https://www.casasymas.com.uy/propiedades/venta/casa/montevideo"

paginas = range(1, 60)
all_links = []

for i in paginas:
    url = f"{base_url}/pagina-{i}"
    print(f"Scrapeando: {url}")
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")

    # Buscar tarjetas con la clase "card prop-entrada"
    tarjetas = soup.select("a.card.prop-entrada")

    # Extraer los hrefs
    links = [a['href'] for a in tarjetas if a.has_attr("href") and len(a['href']) > 10]
    links = list(set(links))  # eliminar duplicados

    all_links.extend(links)

final_links = [f"https://www.casasymas.com.uy{link}" if link.startswith("/") else link for link in all_links]

# Mostrar resultados
print("\n LINKS SCRAPEADOS:")
for link in final_links:
    print(link)
    
print(f"Total: {len(final_links)}")


Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-1
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-2
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-3
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-4
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-5
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-6
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-7
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-8
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-9
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-10
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-11
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo

## Scrapeamos los links

Scrapping de los links para obtener columnas relevantes como precio y caracteristicas

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime
import os
import sys

datos = []

for url in final_links:
    print(f"Procesando: {url}")
    try:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content, "html.parser")

        precio = soup.select_one(".rate-info")
        precio = precio.get_text(strip=True) if precio else None

        caracteristicas = {}
        li_tags = soup.select("div.details-info li")
        
        for li in li_tags:
            img = li.find("img", class_="iconos")
            if img and img.has_attr("title"):
                key = img["title"].strip()
                text = li.get_text(strip=True)
                
                value = text.replace(f"{key}:", "").strip()
                caracteristicas[key] = value

        datos.append({
            "url": url,
            "precio": precio,
            **caracteristicas
        })
        
        time.sleep(1)

    except Exception as e:
        print(f"Error en {url}: {e}")

df = pd.DataFrame(datos)
print(df.head())

base_path = "datasets"
title_csv = "propiedades_scrapeadas_" + datetime.now().strftime("%Y-%m-%d") + ".csv"

full_path = os.path.join(base_path, title_csv)

# exportar a csv

df.to_csv(full_path, index=False)
print(f"Archivo guardado como {title_csv}.csv en datasets/")


Procesando: https://www.casasymas.com.uy/propiedad/222586-casa-en-venta-de-5-dormitorios-en-punta-gorda-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/194576-venta-casa-4-dormitorios-servicio-punta-gorda-ref-1879
Procesando: https://www.casasymas.com.uy/propiedad/186603-casa-en-venta-de-4-dormitorios-en-parque-batlle-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/227830-casa-en-venta-c-cochera-en-punta-carretas
Procesando: https://www.casasymas.com.uy/propiedad/212637-casa-en-venta-de-2-dormitorios-en-belvedere
Procesando: https://www.casasymas.com.uy/propiedad/225135-venta-casa-4-dormitorios-3-banos-y-cochera-en-carrasco-sur-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/225720-venta-casa-3-dormitorios-carrasco
Procesando: https://www.casasymas.com.uy/propiedad/90237-casa-en-venta-de-2-dormitorios-en-carrasco-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/186448-casa-en-venta-de-4-dormitorios-en-pocitos-montevideo
Procesando:

## Merge Datasets

Como son pocos datos para trabajar con una sola corrida, se hicieron varias corridas y se mergearon los datos de ambos datasets. 

In [ ]:
import pandas as pd
import os

datasets_path = "datasets"

csv_files = [
    "propiedades_scrapeadas_2025-08-21.csv",
    "propiedades_scrapeadas_2025-09-01.csv", 
    "propiedades_scrapeadas_2025-09-08.csv"
]

dataframes = [df]

for csv_file in csv_files:
    file_path = os.path.join(datasets_path, csv_file)
    try:
        temp_df = pd.read_csv(file_path)
        dataframes.append(temp_df)
        print(f"Loaded: {csv_file} - {len(temp_df)} rows")
    except Exception as e:
        print(f"Error loading {csv_file}: {e}")

merged_df = pd.concat(dataframes, ignore_index=True)

print(f"\nTotal rows before removing duplicates: {len(merged_df)}")

merged_df = merged_df.drop_duplicates(subset=['url'], keep='first')

print(f"Total rows after removing duplicates: {len(merged_df)}")
print(f"Total columns: {len(merged_df.columns)}")

# Display info
print("\nColumns in merged dataset:")
print(merged_df.columns.tolist())

# Save the merged dataset
output_filename = "propiedades_merged_all.csv"
output_path = os.path.join(datasets_path, output_filename)
merged_df.to_csv(output_path, index=False)

print(f"\nMerged dataset saved as: {output_filename}")

✅ Loaded: propiedades_scrapeadas_2025-08-21.csv - 1635 rows
✅ Loaded: propiedades_scrapeadas_2025-09-01.csv - 1853 rows
✅ Loaded: propiedades_scrapeadas_2025-09-08.csv - 1934 rows

📊 Total rows before removing duplicates: 7208
📊 Total rows after removing duplicates: 2446
📊 Total columns: 32

📋 Columns in merged dataset:
['url', 'precio', 'Dormitorios', 'Baños', 'Superficie', 'Superficie Construida', 'Superficie Terreno', 'Dormitorios Servicio', 'Garage', 'Calefacción', 'Cocina', 'Año de Construcción', 'Jardín', 'Dormitorios Suite', 'Piscina', 'Orientación', 'Plantas', 'Parrillero', 'Acepta Mascotas', 'Superficie Balcones', 'Disposición', 'Pisos Edificio', 'Lavadero', 'Seguridad', 'Altura', 'Aptos por piso', 'Patio', 'Muebles', 'Propiedad Horizontal', 'Gastos Comúnes', 'Piso', 'Ascensor']

👀 First few rows:
                                                 url            precio  \
0  https://www.casasymas.com.uy/propiedad/222586-...  U$S 580.000Venta   
1  https://www.casasymas.com.uy/pr